In [18]:
#importing libraries 
import math 
import string
import sksurv
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
#import models 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

from sklearn import set_config
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sksurv.linear_model import CoxnetSurvivalAnalysis, CoxPHSurvivalAnalysis
from sklearn.model_selection import GridSearchCV, KFold
from sksurv.preprocessing import OneHotEncoder

set_config(display="text")  # displays text representation of estimators


In [19]:
#load data into this nb
df_train = pd.read_csv('train.csv')
df_train


,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,...,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,efs,efs_time
0,0,N/A - non-malignant indication,No,NaN,No,NaN,NaN,No TBI,No,6.0,...,NaN,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,42.356
1,1,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,...,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0,1.0,4.672
2,2,N/A - non-malignant indication,No,NaN,No,2.0,8.0,No TBI,No,6.0,...,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,19.793
3,3,High,No,Intermediate,No,2.0,8.0,No TBI,No,6.0,...,Permissive mismatched,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,102.349
4,4,High,No,NaN,No,2.0,8.0,No TBI,No,6.0,...,Permissive mismatched,Related,MEL,8.0,No,2.0,No,10.0,0.0,16.223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28795,28795,Intermediate - TED AML case <missing cytogenetics,NaN,Favorable,No,2.0,8.0,No TBI,No,6.0,...,Bi-directional non-permissive,NaN,"N/A, Mel not given",8.0,NaN,2.0,No,10.0,0.0,18.633
28796,28796,High,No,Poor,Yes,1.0,4.0,No TBI,No,5.0,...,GvH non-permissive,Related,"N/A, Mel not given",6.0,Yes,1.0,Yes,8.0,1.0,4.892
28797,28797,TBD cytogenetics,NaN,Poor,NaN,2.0,8.0,No TBI,NaN,6.0,...,GvH non-permissive,Unrelated,"N/A, Mel not given",8.0,NaN,2.0,No,10.0,0.0,23.157
28798,28798,N/A - non-malignant indication,No,Poor,No,1.0,4.0,No TBI,No,3.0,...,Permissive mismatched,Related,MEL,4.0,No,1.0,No,5.0,0.0,52.351


In [20]:
#dropping rows with more than a third missing data 
df_dropped_na = pd.DataFrame(df_train.dropna(thresh=40))

# Creating target dataframe
efs = df_dropped_na["efs"]
efs = efs.astype(bool)
efs_time = df_dropped_na["efs_time"]

dtype = [('efs', 'bool'), ('efs_time', 'float64')]  

# Create the structured array
efs_vars = efs_vars = np.array(list(zip(efs, efs_time)), dtype=dtype)
print(efs_vars)


#drop efs_time and efs from df_train as this are the targets
df_dropped_na.drop("efs", axis=1, inplace=True)
df_dropped_na.drop("efs_time", axis=1, inplace=True)

[(False, 42.356) ( True,  4.672) (False, 19.793) ... (False, 23.157)
 (False, 52.351) (False, 25.158)]


In [21]:


# Change categorical nan values to other 

df_dropped_na[["dri_score", "psych_disturb", "cyto_score", "diabetes", "tbi_status", "arrhythmia", "graft_type", "vent_hist", "renal_issue", "pulm_severe", 
"prim_disease_hct", "cmv_status", "tce_imm_match", "rituximab", "prod_type", "cyto_score_detail", "conditioning_intensity", "ethnicity", "obesity", "mrd_hct",
"in_vivo_tcd", "tce_match", "hepatic_severe", "prior_tumor", "peptic_ulcer", "gvhd_proph", "rheum_issue", "sex_match", "race_group", "hepatic_mild", "tce_div_match",
"donor_related", "melphalan_dose", "cardiac", "pulm_moderate"]] = df_dropped_na[["dri_score", "psych_disturb", "cyto_score", "diabetes", "tbi_status", "arrhythmia", "graft_type", "vent_hist", "renal_issue", "pulm_severe", 
"prim_disease_hct", "cmv_status", "tce_imm_match", "rituximab", "prod_type", "cyto_score_detail", "conditioning_intensity", "ethnicity", "obesity", "mrd_hct",
"in_vivo_tcd", "tce_match", "hepatic_severe", "prior_tumor", "peptic_ulcer", "gvhd_proph", "rheum_issue", "sex_match", "race_group", "hepatic_mild", "tce_div_match",
"donor_related", "melphalan_dose", "cardiac", "pulm_moderate"]].fillna(value="other")


#one hot encode 
 
print(isinstance(df_dropped_na, pd.DataFrame))
#change from obj to category 

df_dropped_na[["dri_score", "psych_disturb", "cyto_score", "diabetes", "tbi_status", "arrhythmia", "graft_type", "vent_hist", "renal_issue", "pulm_severe", 
"prim_disease_hct", "cmv_status", "tce_imm_match", "rituximab", "prod_type", "cyto_score_detail", "conditioning_intensity", "ethnicity", "obesity", "mrd_hct",
"in_vivo_tcd", "tce_match", "hepatic_severe", "prior_tumor", "peptic_ulcer", "gvhd_proph", "rheum_issue", "sex_match", "race_group", "hepatic_mild", "tce_div_match",
"donor_related", "melphalan_dose", "cardiac", "pulm_moderate"]] =  df_dropped_na[["dri_score", "psych_disturb", "cyto_score", "diabetes", "tbi_status", "arrhythmia", "graft_type", "vent_hist", "renal_issue", "pulm_severe", 
"prim_disease_hct", "cmv_status", "tce_imm_match", "rituximab", "prod_type", "cyto_score_detail", "conditioning_intensity", "ethnicity", "obesity", "mrd_hct",
"in_vivo_tcd", "tce_match", "hepatic_severe", "prior_tumor", "peptic_ulcer", "gvhd_proph", "rheum_issue", "sex_match", "race_group", "hepatic_mild", "tce_div_match",
"donor_related", "melphalan_dose", "cardiac", "pulm_moderate"]].astype('category')
#encode
Encoder = OneHotEncoder()
df_encoded = Encoder.fit_transform(df_dropped_na)



#drop "other" columns as they represent NAs
df_encoded = df_encoded[df_encoded.columns.drop(list(df_encoded.filter(regex='other')))]
df_encoded
                                                        

True


,ID,dri_score=High - TED AML case <missing cytogenetics,dri_score=Intermediate,dri_score=Intermediate - TED AML case <missing cytogenetics,dri_score=Low,dri_score=Missing disease status,dri_score=N/A - disease not classifiable,dri_score=N/A - non-malignant indication,dri_score=N/A - pediatric,dri_score=TBD cytogenetics,...,donor_related=Related,donor_related=Unrelated,"melphalan_dose=N/A, Mel not given",hla_low_res_8,cardiac=Not done,cardiac=Yes,hla_match_drb1_high,pulm_moderate=Not done,pulm_moderate=Yes,hla_low_res_10
0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,1.0,8.0,0.0,0.0,2.0,0.0,0.0,10.0
1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,8.0,0.0,0.0,2.0,0.0,1.0,10.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,8.0,0.0,0.0,2.0,0.0,0.0,10.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,8.0,0.0,0.0,2.0,0.0,0.0,10.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,8.0,0.0,0.0,2.0,0.0,0.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28795,28795,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,8.0,0.0,0.0,2.0,0.0,0.0,10.0
28796,28796,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,6.0,0.0,1.0,1.0,0.0,1.0,8.0
28797,28797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,8.0,0.0,0.0,2.0,0.0,0.0,10.0
28798,28798,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,5.0


In [22]:


#using KNN to fill in missing numerical values 
imputer = KNNImputer(n_neighbors = 165, weights='distance') 
df_imputed = pd.DataFrame(imputer.fit_transform(df_encoded), columns=df_encoded.columns)
 


In [23]:
# Feature select

# Feature Selection -> Cox
def fit_and_score_features(X, y):
    n_features = X.shape[1]
    scores = np.empty(n_features)
    m = CoxPHSurvivalAnalysis()
    for j in range(n_features):
        Xj = X[:, j : j + 1]
        m.fit(Xj, y)
        scores[j] = m.score(Xj, y)
    return scores

#output here is the rank of feature's ability to predict outcome
scores = fit_and_score_features(df_imputed.values, efs_vars)
pd.Series(scores, index=df_imputed.columns).sort_values(ascending=False)

age_at_hct                                     0.579348
comorbidity_score                              0.563828
prod_type=PB                                   0.560543
graft_type=Peripheral blood                    0.559526
conditioning_intensity=RIC                     0.550620
                                                 ...   
rituximab=Yes                                  0.499894
tbi_status=TBI +- Other, -cGy, unknown dose    0.499882
hla_match_b_high                               0.497155
hla_match_c_low                                0.497099
dri_score=Intermediate                         0.494388
Length: 140, dtype: float64

In [24]:
%%time
#here we're picking how many features to use in model
pipe = Pipeline(
    [
        ("encode", OneHotEncoder()),
        ("select", SelectKBest(fit_and_score_features, k=3)),
        ("model", CoxPHSurvivalAnalysis()),
    ]
)

param_grid = {"select__k": np.arange(35, 80, 5)}
cv = KFold(n_splits=3, random_state=1, shuffle=True) #potential hyperparamters to mess w/
gcv = GridSearchCV(pipe, param_grid, return_train_score=True, cv=cv)
gcv.fit(df_imputed, efs_vars)

results = pd.DataFrame(gcv.cv_results_).sort_values(by="mean_test_score", ascending=False)
results.loc[:, ~results.columns.str.endswith("_time")]



CPU times: total: 2h 35s
Wall time: 2h 2min 2s


,param_select__k,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
8,75,{'select__k': 75},0.655838,0.657163,0.651026,0.654676,0.002637,1,0.657816,0.658983,0.661382,0.659394,0.001484
7,70,{'select__k': 70},0.656354,0.656104,0.650365,0.654274,0.002766,2,0.657457,0.657827,0.660191,0.658492,0.001211
6,65,{'select__k': 65},0.656377,0.656362,0.650070,0.654270,0.002970,3,0.656524,0.657543,0.659364,0.657810,0.001175
5,60,{'select__k': 60},0.655485,0.655625,0.650178,0.653763,0.002536,4,0.655844,0.656299,0.658067,0.656737,0.000959
4,55,{'select__k': 55},0.654313,0.654557,0.650085,0.652985,0.002053,5,0.654803,0.655581,0.656886,0.655757,0.000859
3,50,{'select__k': 50},0.653237,0.654274,0.649252,0.652255,0.002165,6,0.654171,0.654720,0.655725,0.654872,0.000644
2,45,{'select__k': 45},0.653323,0.651441,0.648263,0.651009,0.002088,7,0.654062,0.651636,0.654598,0.653432,0.001289
1,40,{'select__k': 40},0.649992,0.648761,0.645953,0.648235,0.001690,8,0.649583,0.649977,0.651898,0.650486,0.001011
0,35,{'select__k': 35},0.647981,0.647920,0.645252,0.647051,0.001272,9,0.647026,0.649485,0.650287,0.648933,0.001388


In [25]:
%%time
pipe.set_params(**gcv.best_params_)
pipe.fit(df_imputed, efs_vars)

encoder, transformer, final_estimator = (s[1] for s in pipe.steps)
pd.Series(final_estimator.coef_, index=encoder.encoded_columns_[transformer.get_support()])

CPU times: total: 7min 10s
Wall time: 7min 13s


dri_score=High - TED AML case <missing cytogenetics   -0.049966
dri_score=N/A - non-malignant indication              -0.121621
dri_score=N/A - pediatric                             -0.101137
dri_score=TBD cytogenetics                            -0.070487
psych_disturb=Yes                                      0.044755
                                                         ...   
hla_low_res_8                                         -0.103151
cardiac=Yes                                            0.271738
hla_match_drb1_high                                    0.219973
pulm_moderate=Yes                                      0.049171
hla_low_res_10                                         0.007668
Length: 75, dtype: float64

In [26]:
print(encoder.encoded_columns_[transformer.get_support()])

Index(['dri_score=High - TED AML case <missing cytogenetics',
       'dri_score=N/A - non-malignant indication', 'dri_score=N/A - pediatric',
       'dri_score=TBD cytogenetics', 'psych_disturb=Yes',
       'cyto_score=Intermediate', 'cyto_score=Poor', 'cyto_score=TBD',
       'diabetes=Yes', 'hla_high_res_8', 'tbi_status=TBI +- Other, <=cGy',
       'tbi_status=TBI +- Other, >cGy', 'arrhythmia=Yes', 'hla_low_res_6',
       'graft_type=Peripheral blood', 'pulm_severe=Yes',
       'prim_disease_hct=ALL', 'prim_disease_hct=AML', 'prim_disease_hct=HIS',
       'prim_disease_hct=IEA', 'prim_disease_hct=IIS', 'prim_disease_hct=IPA',
       'prim_disease_hct=MDS', 'prim_disease_hct=MPN',
       'prim_disease_hct=Other leukemia', 'prim_disease_hct=PCD',
       'prim_disease_hct=SAA', 'hla_high_res_6', 'cmv_status=+/-',
       'cmv_status=-/+', 'cmv_status=-/-', 'hla_high_res_10',
       'tce_imm_match=H/H', 'tce_imm_match=P/P', 'hla_match_drb1_low',
       'prod_type=PB', 'cyto_score_detail=I

In [27]:
%%time
#actually predict 
final_estimator = gcv.best_estimator_.named_steps["model"]
predicted_hazards = final_estimator.predict(pd.DataFrame(df_imputed[['dri_score=High - TED AML case <missing cytogenetics',
       'dri_score=N/A - non-malignant indication', 'dri_score=N/A - pediatric',
       'dri_score=TBD cytogenetics', 'psych_disturb=Yes',
       'cyto_score=Intermediate', 'cyto_score=Poor', 'cyto_score=TBD',
       'diabetes=Yes', 'hla_high_res_8', 'tbi_status=TBI +- Other, <=cGy',
       'tbi_status=TBI +- Other, >cGy', 'arrhythmia=Yes', 'hla_low_res_6',
       'graft_type=Peripheral blood', 'pulm_severe=Yes',
       'prim_disease_hct=ALL', 'prim_disease_hct=AML', 'prim_disease_hct=HIS',
       'prim_disease_hct=IEA', 'prim_disease_hct=IIS', 'prim_disease_hct=IPA',
       'prim_disease_hct=MDS', 'prim_disease_hct=MPN',
       'prim_disease_hct=Other leukemia', 'prim_disease_hct=PCD',
       'prim_disease_hct=SAA', 'hla_high_res_6', 'cmv_status=+/-',
       'cmv_status=-/+', 'cmv_status=-/-', 'hla_high_res_10',
       'tce_imm_match=H/H', 'tce_imm_match=P/P', 'hla_match_drb1_low',
       'prod_type=PB', 'cyto_score_detail=Intermediate',
       'cyto_score_detail=Poor', 'cyto_score_detail=TBD',
       'conditioning_intensity=NMA', 'conditioning_intensity=RIC',
       'ethnicity=Not Hispanic or Latino', 'year_hct', 'obesity=Yes',
       'in_vivo_tcd=Yes', 'tce_match=Permissive', 'hla_match_a_high',
       'hepatic_severe=Yes', 'donor_age', 'prior_tumor=Not done',
       'prior_tumor=Yes', 'hla_match_b_low', 'age_at_hct', 'hla_match_a_low',
       'gvhd_proph=Cyclophosphamide alone', 'gvhd_proph=FKalone',
       'gvhd_proph=Other GVHD Prophylaxis', 'sex_match=F-M', 'sex_match=M-F',
       'sex_match=M-M', 'race_group=Asian',
       'race_group=Black or African-American', 'race_group=More than one race',
       'race_group=White', 'comorbidity_score', 'karnofsky_score',
       'tce_div_match=Permissive mismatched', 'donor_related=Related',
       'donor_related=Unrelated', 'melphalan_dose=N/A, Mel not given',
       'hla_low_res_8', 'cardiac=Yes', 'hla_match_drb1_high',
       'pulm_moderate=Yes', 'hla_low_res_10']]))
predicted_hazards = predicted_hazards * -1

CPU times: total: 15.6 ms
Wall time: 15.1 ms


C:\Users\jdhru\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but CoxPHSurvivalAnalysis was fitted without feature names
  warnings.warn(


In [28]:
print(predicted_hazards)

[81.88177283 80.47212307 82.3873716  ... 79.87340177 81.17599352
 81.63274789]


In [29]:
%%time
#c-index score of predictions
from lifelines.utils import concordance_index
# Actual survival times and event indicators

# Calculate the concordance index
c_index = concordance_index(efs_time, pd.DataFrame(predicted_hazards), efs)
print(f"Concordance Index: {c_index}")

Concordance Index: 0.6584468526977052
CPU times: total: 469 ms
Wall time: 528 ms


In [30]:
#now we load in test data and do risk score predictions 
test_data = pd.read_csv('test.csv')
test_data

,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,...,karnofsky_score,hepatic_mild,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10
0,28800,N/A - non-malignant indication,No,NaN,No,NaN,NaN,No TBI,No,6.0,...,90.0,No,NaN,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0
1,28801,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,...,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0
2,28802,N/A - non-malignant indication,No,NaN,No,2.0,8.0,No TBI,No,6.0,...,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0


In [31]:
# Change categorical nan values to other 

test_data[["dri_score", "psych_disturb", "cyto_score", "diabetes", "tbi_status", "arrhythmia", "graft_type", "vent_hist", "renal_issue", "pulm_severe", 
"prim_disease_hct", "cmv_status", "tce_imm_match", "rituximab", "prod_type", "cyto_score_detail", "conditioning_intensity", "ethnicity", "obesity", "mrd_hct",
"in_vivo_tcd", "tce_match", "hepatic_severe", "prior_tumor", "peptic_ulcer", "gvhd_proph", "rheum_issue", "sex_match", "race_group", "hepatic_mild", "tce_div_match",
"donor_related", "melphalan_dose", "cardiac", "pulm_moderate"]] = test_data[["dri_score", "psych_disturb", "cyto_score", "diabetes", "tbi_status", "arrhythmia", "graft_type", "vent_hist", "renal_issue", "pulm_severe", 
"prim_disease_hct", "cmv_status", "tce_imm_match", "rituximab", "prod_type", "cyto_score_detail", "conditioning_intensity", "ethnicity", "obesity", "mrd_hct",
"in_vivo_tcd", "tce_match", "hepatic_severe", "prior_tumor", "peptic_ulcer", "gvhd_proph", "rheum_issue", "sex_match", "race_group", "hepatic_mild", "tce_div_match",
"donor_related", "melphalan_dose", "cardiac", "pulm_moderate"]].fillna(value="other")


#one hot encode 
 
#change from obj to category 

test_data[["dri_score", "psych_disturb", "cyto_score", "diabetes", "tbi_status", "arrhythmia", "graft_type", "vent_hist", "renal_issue", "pulm_severe", 
"prim_disease_hct", "cmv_status", "tce_imm_match", "rituximab", "prod_type", "cyto_score_detail", "conditioning_intensity", "ethnicity", "obesity", "mrd_hct",
"in_vivo_tcd", "tce_match", "hepatic_severe", "prior_tumor", "peptic_ulcer", "gvhd_proph", "rheum_issue", "sex_match", "race_group", "hepatic_mild", "tce_div_match",
"donor_related", "melphalan_dose", "cardiac", "pulm_moderate"]] =  test_data[["dri_score", "psych_disturb", "cyto_score", "diabetes", "tbi_status", "arrhythmia", "graft_type", "vent_hist", "renal_issue", "pulm_severe", 
"prim_disease_hct", "cmv_status", "tce_imm_match", "rituximab", "prod_type", "cyto_score_detail", "conditioning_intensity", "ethnicity", "obesity", "mrd_hct",
"in_vivo_tcd", "tce_match", "hepatic_severe", "prior_tumor", "peptic_ulcer", "gvhd_proph", "rheum_issue", "sex_match", "race_group", "hepatic_mild", "tce_div_match",
"donor_related", "melphalan_dose", "cardiac", "pulm_moderate"]].astype('category')
#encode
df_encoded = Encoder.transform(test_data)



#drop "other" column
df_encoded = df_encoded[df_encoded.columns.drop(list(df_encoded.filter(regex='other')))]
df_encoded

,ID,dri_score=High - TED AML case <missing cytogenetics,dri_score=Intermediate,dri_score=Intermediate - TED AML case <missing cytogenetics,dri_score=Low,dri_score=Missing disease status,dri_score=N/A - disease not classifiable,dri_score=N/A - non-malignant indication,dri_score=N/A - pediatric,dri_score=TBD cytogenetics,...,donor_related=Related,donor_related=Unrelated,"melphalan_dose=N/A, Mel not given",hla_low_res_8,cardiac=Not done,cardiac=Yes,hla_match_drb1_high,pulm_moderate=Not done,pulm_moderate=Yes,hla_low_res_10
0,28800,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,1.0,8.0,0.0,0.0,2.0,0.0,0.0,10.0
1,28801,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,8.0,0.0,0.0,2.0,0.0,1.0,10.0
2,28802,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,8.0,0.0,0.0,2.0,0.0,0.0,10.0


In [32]:
#KNN for missing values 

#n_neighbors rule of thumb is the sqrt of the samples
imputer = KNNImputer(n_neighbors = int(np.sqrt(test_data.shape[0])) , weights='distance') 
df_imputed = pd.DataFrame(imputer.fit_transform(df_encoded), columns=df_encoded.columns)
 

In [33]:
#predict using model 

#actually predict 
final_estimator = gcv.best_estimator_.named_steps["model"]
predicted_hazards = final_estimator.predict(pd.DataFrame(df_imputed[['dri_score=High - TED AML case <missing cytogenetics',
       'dri_score=N/A - non-malignant indication', 'dri_score=N/A - pediatric',
       'dri_score=TBD cytogenetics', 'psych_disturb=Yes',
       'cyto_score=Intermediate', 'cyto_score=Poor', 'cyto_score=TBD',
       'diabetes=Yes', 'hla_high_res_8', 'tbi_status=TBI +- Other, <=cGy',
       'tbi_status=TBI +- Other, >cGy', 'arrhythmia=Yes', 'hla_low_res_6',
       'graft_type=Peripheral blood', 'pulm_severe=Yes',
       'prim_disease_hct=ALL', 'prim_disease_hct=AML', 'prim_disease_hct=HIS',
       'prim_disease_hct=IEA', 'prim_disease_hct=IIS', 'prim_disease_hct=IPA',
       'prim_disease_hct=MDS', 'prim_disease_hct=MPN',
       'prim_disease_hct=Other leukemia', 'prim_disease_hct=PCD',
       'prim_disease_hct=SAA', 'hla_high_res_6', 'cmv_status=+/-',
       'cmv_status=-/+', 'cmv_status=-/-', 'hla_high_res_10',
       'tce_imm_match=H/H', 'tce_imm_match=P/P', 'hla_match_drb1_low',
       'prod_type=PB', 'cyto_score_detail=Intermediate',
       'cyto_score_detail=Poor', 'cyto_score_detail=TBD',
       'conditioning_intensity=NMA', 'conditioning_intensity=RIC',
       'ethnicity=Not Hispanic or Latino', 'year_hct', 'obesity=Yes',
       'in_vivo_tcd=Yes', 'tce_match=Permissive', 'hla_match_a_high',
       'hepatic_severe=Yes', 'donor_age', 'prior_tumor=Not done',
       'prior_tumor=Yes', 'hla_match_b_low', 'age_at_hct', 'hla_match_a_low',
       'gvhd_proph=Cyclophosphamide alone', 'gvhd_proph=FKalone',
       'gvhd_proph=Other GVHD Prophylaxis', 'sex_match=F-M', 'sex_match=M-F',
       'sex_match=M-M', 'race_group=Asian',
       'race_group=Black or African-American', 'race_group=More than one race',
       'race_group=White', 'comorbidity_score', 'karnofsky_score',
       'tce_div_match=Permissive mismatched', 'donor_related=Related',
       'donor_related=Unrelated', 'melphalan_dose=N/A, Mel not given',
       'hla_low_res_8', 'cardiac=Yes', 'hla_match_drb1_high',
       'pulm_moderate=Yes', 'hla_low_res_10']]))
predicted_hazards = predicted_hazards * -1

C:\Users\jdhru\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but CoxPHSurvivalAnalysis was fitted without feature names
  warnings.warn(


In [34]:
#make submission file 

# Creating a new DataFrame by combining the 'ID' column with the list of prediected hazard
submission_df = pd.DataFrame({
    'ID': test_data['ID'],
    'prediction': predicted_hazards }) 

# Output to CSV without the additional index column
submission_df.to_csv('submission.csv', index=False)

print(submission_df)



      ID  prediction
0  28800   81.810365
1  28801   80.472123
2  28802   82.336962
